# Sample Script v2.0

In [21]:
import keras
import os
import numpy as np

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

from MyUtils.Dataset import DatasetLoader
from MyUtils.ResnetBuilder import ResnetBuilder

from keras.optimizers import Adam

In [24]:
ROOT_DIR='/Users/abalaji/mydata/DataScience/TinyImageNet'
IMAGE_ROOT_DIR=os.path.join(ROOT_DIR, 'tiny-imagenet-200')

# utilities.
UTILS_ROOT_DIR='/Users/abalaji/mydata/EIP_3_0/Session 4/Assignment'
MY_UTILS_DIR= os.path.join(UTILS_ROOT_DIR,'MyUtils')
SUPPORT_FILES_DIR=os.path.join(MY_UTILS_DIR, 'SupportFiles')

#IMAGE_SIZE=64
#NUM_CHANNELS=3

## for training
NUM_CLASSES_TO_TRAIN_AND_VAL=2

# Hyperparameters
batch_size = 50
epochs = 50

In [11]:
## set up the data loader
dataLoader = DatasetLoader(IMAGE_ROOT_DIR)
dataLoader.set_support_files_dir(SUPPORT_FILES_DIR)

## get all class ids and their names.
(class_id_arr, class_names_arr) = dataLoader.get_class_info()

## load training images
(training_images, training_label_ids) = dataLoader.load_training_images_for_a_list_classes(
                                                class_id_arr[0:NUM_CLASSES_TO_TRAIN_AND_VAL], 
                                                resize_flag=True)

(val_images, val_label_ids) = dataLoader.load_validation_images_for_a_list_classes(
                                                class_id_arr[0:NUM_CLASSES_TO_TRAIN_AND_VAL], 
                                                resize_flag=True)

# shuffle the training data
np.random.seed(123)
shuffle_index = np.random.permutation(len(training_label_ids))
training_images = training_images[shuffle_index]
training_label_ids = training_label_ids[shuffle_index]

In [12]:
le = LabelEncoder()
training_le = le.fit(training_label_ids)
training_labels_encoded = training_le.transform(training_label_ids)

## validation
val_labels_encoded = training_le.transform(val_label_ids)

# convert to one hot encoing - it DOES NOT accept strings.
training_labels = keras.utils.to_categorical(training_labels_encoded, NUM_CLASSES_TO_TRAIN_AND_VAL)
val_labels  = keras.utils.to_categorical(val_labels_encoded, NUM_CLASSES_TO_TRAIN_AND_VAL)


In [19]:
img_height, img_width, num_channel = training_images.shape[1],training_images.shape[2],training_images.shape[3]

x_train = training_images
y_train = training_labels

x_val   = val_images
y_val   = val_labels

In [14]:
print(img_height)
print(img_width)
print(num_channel)

32
32
3


In [15]:
# num_channels, rows, cols
input_shape = (num_channel,img_height,img_width)
num_classes = NUM_CLASSES_TO_TRAIN_AND_VAL
model = ResnetBuilder.build_resnet_50(input_shape, num_classes)

In [18]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 16, 16, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 16, 16, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 16, 16, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [22]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

## to save the best model 
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('tiny_best_model.hdf5', monitor='val_acc', verbose=1, save_best_only=True)

callbacks = [checkpoint]

In [26]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=callbacks)

Train on 1000 samples, validate on 100 samples
Epoch 1/50
1000/1000 [==============================] - 50s 50ms/step - loss: 6.0932 - acc: 0.6670 - val_loss: 6.4880 - val_acc: 0.6000

Epoch 00001: val_acc improved from -inf to 0.60000, saving model to tiny_best_model.hdf5
Epoch 2/50
1000/1000 [==============================] - 35s 35ms/step - loss: 5.2414 - acc: 0.8110 - val_loss: 5.3490 - val_acc: 0.7000

Epoch 00002: val_acc improved from 0.60000 to 0.70000, saving model to tiny_best_model.hdf5
Epoch 3/50
 500/1000 [==============>...............] - ETA: 17s - loss: 4.8403 - acc: 0.9040

KeyboardInterrupt: 